# Ejercicio Covid-19

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Fechas de hoy(datos España) y ayer (datos inetrnacionales)
from datetime import datetime, timedelta
from datetime import date
date_today = date.today()
date_today_complete = date.today()
date_daybefore = date_today - timedelta(days=1)
date_today = date_today.strftime('%d/%m')
date_daybefore = date_daybefore.strftime('%d/%m')

In [3]:
#Importante revisar si es CCAA recoge bien los datos de CCAA, se han hecho cambios para el 14-04, porque no los cogía bien
new_inform = 77
new_before = new_inform - 1
path_spain = 'pdf_files/Actualizacion_77_COVID-19.pdf'
path_spain_before = 'pdf_files/Actualizacion_76_COVID-19.pdf'

date_today_large = date_today_complete.strftime('%Y-%m-%d')
date_today_large

'2020-04-16'

# Preparación inicial

## Descargar PDFs del Ministerio de Sanidad

In [4]:
#Importar PDFs
import os
import requests
from bs4 import BeautifulSoup
import re

#Función para descargar PDFs
def download_file(url):
    local_filename = url.split('/')[-1]
    # NOTE the stream=True parameter below
    with requests.get(url, stream=True) as r:
        r.raise_for_status()
        with open(local_filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=8192): 
                if chunk: # filter out keep-alive new chunks
                    f.write(chunk)
                    # f.flush()
    print(local_filename)
    return local_filename

#
DAILY_FOLDER = "./pdf_files/"
ES_REPORT_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/situacionActual.htm"
PDF_BASE_URL = "https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/"

#Función para revisar los nombres de archivos
def get_file_names(url):
    try:
        req_page = requests.get(url, verify=True)
    except Exception as e:
        raise Exception(f"Could not get web page content: {e}")
    
    re_pdf = re.compile(r"(documentos/.+?\.pdf)")

    pdf_paths = list(set(re_pdf.findall(req_page.text)))
    return pdf_paths

#Reviso los nomnbres de archivos
pdf_paths = get_file_names(ES_REPORT_URL)

pdf_paths = [PDF_BASE_URL+i for i in pdf_paths if "Informacion_inicial_alerta" not in i]


for pdf_path in pdf_paths:
        pdf_name = pdf_path.split("/")[-1]
        pdf_path_get = requests.get(pdf_path)
        with open(
            os.path.join(DAILY_FOLDER, pdf_name),
            'wb'
        ) as f:
            f.write(pdf_path_get.content)
            
# Descargo PDFs a raíz
for i in range(new_before,new_inform):
    path = 'https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov-China/documentos/Actualizacion_{}_COVID-19.pdf'.format(i+1)
    download_file(path)

# Muevo los archivos a otra carpeta
lst_files = [file for file in os.listdir() if 'Act' in file]
if os.path.isdir("./pdf_files/"):
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))
else:
    os.mkdir("./pdf_data/")
    for f in lst_files:
        os.rename("./{}".format(f), "./pdf_files/{}".format(f))

Actualizacion_77_COVID-19.pdf


## Parsear PDF descargado

In [8]:
# Parsear PDF

from tika import parser
import pandas as pd
from io import StringIO
import re
import tabula

raw = parser.from_file(path_spain)

#Cojo partes del PDF
ind_ini = raw['content'].find('en España:')
ind_fin = raw['content'].find('mundo:')


#Coger fecha del PDF
def get_fecha(string):
    ind_ini = string.find('(COVID-19)')
    ind_fin = string.find('.20')
    print(string[ind_ini:ind_fin])
    return string[ind_ini:ind_fin].split('\n')[-1]+'.2020'

fecha_data_spain = get_fecha(raw['content'])
print(fecha_data_spain)

# Coger tablar primera página
import tabula
from tabula import read_pdf
data_spain_ccaa_all = tabula.read_pdf(path_spain, stream=True, pages=1)[0]
data_spain_ccaa_all.columns = ['Comunidades Autónomas','Confirmados', 'IA', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']

'''
# VERSIÓN DEL 14-04-20 para que cuadre bien
data_spain_ccaa = data_spain_ccaa_all.iloc[4:27]
data_spain_ccaa = data_spain_ccaa.drop([data_spain_ccaa.index[6], data_spain_ccaa.index[8], data_spain_ccaa.index[14], data_spain_ccaa.index[16]])
data_spain_ccaa.loc[data_spain_ccaa['Comunidades Autónomas'] == 'Mancha', ['Comunidades Autónomas']] = 'Castilla La Mancha'
'''

# La versión normal:
#data_spain_ccaa = data_spain_ccaa_all.iloc[3:22]
# OJO!!!!! Depende del día cambia la posición!!!

# VERSIÓN DEL 15-04-20 para que cuadre bien
data_spain_ccaa = data_spain_ccaa_all.iloc[4:23]
# OJO!!!!! Depende del día cambia la posición!!!

# Cambio las columnas
data_spain_ccaa = data_spain_ccaa[['Comunidades Autónomas', 'Confirmados', 'Curados', 'Fallecidos', 'Nuevos casos (24h)']]

#Quitar símbolos raros qye nos sobran
data_spain_ccaa['Confirmados'] = data_spain_ccaa['Confirmados'].str.replace('.','')
data_spain_ccaa['Fallecidos'] = data_spain_ccaa['Fallecidos'].str.replace('.','')
data_spain_ccaa['Curados'] = data_spain_ccaa['Curados'].str.replace('.','')
data_spain_ccaa['Nuevos casos (24h)'] = data_spain_ccaa['Nuevos casos (24h)'].str.replace('.','')
#Añadido el 15-04
data_spain_ccaa['Confirmados'] = data_spain_ccaa['Confirmados'].str.replace('*','')
data_spain_ccaa['Nuevos casos (24h)'] = data_spain_ccaa['Nuevos casos (24h)'].str.replace('*','')

# Seleccionar índice Comunidades Autónomas
#data_spain_ccaa = data_spain_ccaa.set_index('Comunidades Autónomas')
data_spain_ccaa

(COVID-19). 16.04
(COVID-19). 16.04.2020


ValueError: Length mismatch: Expected axis has 6 elements, new values have 8 elements

In [9]:
data_spain_ccaa_all

,CCAA,Unnamed: 0,Unnamed: 1,PCR,Test rápidos de,Unnamed: 2
0,NaN,Total,Nuevos,NaN,NaN,(14 d.)
1,NaN,NaN,NaN,NaN,anticuerpos,NaN
2,Andalucía,10.807,212,10592.0,215,"45,58"
3,Aragón,4.566,228,4411.0,155,"141,44"
4,Asturias,2.170,74,2081.0,89*,"76,85"
5,Baleares,1.637,31,NaN,NaN,"37,67"
6,Canarias,1.988,13,1988.0,NaN,"25,26"
7,Cantabria,1.845,22,NaN,NaN,"99,30"
8,Castilla La Mancha,15.151,471,NaN,NaN,"367,41"
9,Castilla y León,14.380,683,NaN,NaN,"292,76"


# Resumen de datos INICIAL

## Resumen datos globales CSV: covid19_data_drive_resume.csv

In [4]:
# Creación de funciones para sacar datos
def get_data_spain(url):    
    data_spain = pd.read_csv(url)
    data_spain = data_spain.groupby("Country/Region", as_index=False).sum()
    data_spain = data_spain.set_index("Country/Region").transpose()
    data_spain = data_spain['Spain']
    data_spain_actual = int(data_spain.iloc[-1])
    data_spain_before = int(data_spain.iloc[-2])
    data_spain_24h = data_spain_actual - data_spain_before
    return data_spain_24h, data_spain_actual
    
# Función para obtener datos del mundo    
def get_data_world(url):    
    data_world = pd.read_csv(url)
    data_world = data_world.groupby("Country/Region", as_index=False).sum()
    data_world = data_world.set_index("Country/Region").transpose()
    data_world_actual = int(data_world.sum(axis=1).iloc[-1])
    data_world_before = int(data_world.sum(axis=1).iloc[-2])
    data_world_24h = data_world_actual - data_world_before
    return data_world_24h, data_world_actual

# Usar los datos
data_confirmed_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
data_deaths_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
data_recovered_path = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"

# Para España
data_confirmed_spain_24h, data_confirmed_spain_actual = get_data_spain(data_confirmed_path)
data_deaths_spain_24h, data_deaths_spain_actual = get_data_spain(data_deaths_path)
data_recovered_spain_24h, data_recovered_spain_actual = get_data_spain(data_recovered_path)

# Para el mundo
data_confirmed_world_24h, data_confirmed_world_actual = get_data_world(data_confirmed_path)
data_deaths_world_24h, data_deaths_world_actual = get_data_world(data_deaths_path)
data_recovered_world_24h, data_recovered_world_actual = get_data_world(data_recovered_path)

######### Para datos de John Hopkins internacionales

data_spain_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_spain_actual,
                                    'Curados': data_deaths_spain_actual, 
                                    'Fallecidos': data_recovered_spain_actual}, 
                                   index=['Casos totales en España'])

data_spain_newcases = pd.DataFrame({'Confirmados': data_confirmed_spain_24h,
                                    'Curados': data_recovered_spain_24h, 
                                    'Fallecidos': data_deaths_spain_24h}, 
                                   index=['Nuevos casos en España (24h)'])


data_world_newcases = pd.DataFrame({'Confirmados': data_confirmed_world_24h,
                                    'Curados': data_recovered_world_24h, 
                                    'Fallecidos': data_deaths_world_24h}, 
                                   index=['Nuevos casos en el mundo (24h)'])

data_world_resume_actual = pd.DataFrame({'Confirmados': data_confirmed_world_actual,
                                    'Curados': data_recovered_world_actual, 
                                    'Fallecidos': data_deaths_world_actual}, 
                                   index=['Casos totales en el mundo'])

########## Para datos de España del Ministerio de Sanidad
# Coger los datos del día anterior
'''import tabula
from tabula import read_pdf
data_spain_ccaa_daybefore = tabula.read_pdf(path_spain_before, stream=True, pages=1)[0]
data_spain_ccaa_daybefore.columns = ['Comunidades Autónomas','Confirmados', 'IA', 'Hospitalizados', 'UCI', 'Fallecidos', 'Curados', 'Nuevos casos (24h)']

data_spain_ccaa_daybefore =  data_spain_ccaa_daybefore.iloc[22:23].copy()
data_spain_ccaa_daybefore.rename(index={22: "Casos totales en España"}, inplace=True)
data_spain_ccaa_daybefore = data_spain_ccaa_daybefore[['Confirmados', 'Curados', 'Fallecidos']]
data_spain_ccaa_daybefore['Confirmados'] = data_spain_ccaa_daybefore['Confirmados'].str.replace('.','').astype('int')
data_spain_ccaa_daybefore['Fallecidos'] = data_spain_ccaa_daybefore['Fallecidos'].str.replace('.','').astype('int')
data_spain_ccaa_daybefore['Curados'] = data_spain_ccaa_daybefore['Curados'].str.replace('.','').astype('int')

# Coger los datos del día de hoy
data_spain_ccaa_daytoday = data_spain_ccaa_all.iloc[22:23].copy()
data_spain_ccaa_daytoday.rename(index={22: "Casos totales en España"}, inplace=True)
data_spain_ccaa_daytoday = data_spain_ccaa_daytoday[['Confirmados', 'Curados', 'Fallecidos']]
data_spain_ccaa_daytoday['Confirmados'] = data_spain_ccaa_daytoday['Confirmados'].str.replace('.','').astype('int')
data_spain_ccaa_daytoday['Fallecidos'] = data_spain_ccaa_daytoday['Fallecidos'].str.replace('.','').astype('int')
data_spain_ccaa_daytoday['Curados'] = data_spain_ccaa_daytoday['Curados'].str.replace('.','').astype('int')


# Restar datos de dataframes
data_spain_ccaa_daytoday # Muestra los datos del día actual
data_spain_ccaa_daybefore # Muestra los datos del día anterior
data_spain_ccaa_restdays = data_spain_ccaa_daytoday - data_spain_ccaa_daybefore # Muestra los datos en 24h.
data_spain_ccaa_restdays.rename(index={"Casos totales en España": "Nuevos casos en España (24h)"}, inplace=True)
# Aquí fue superimportante ponerlo así, y no así: data_spain_ccaa_restdays.rename(index={0: "Casos totales en España"}, inplace=True), así no funcionaba
'''



########### Junto los datos anteriores en una tabla
data_resume_final = pd.concat([data_world_resume_actual, data_world_newcases, data_spain_resume_actual, data_spain_newcases])

data_resume_final.to_csv('covid19_data_drive_resume.csv', sep = ';')
print(data_resume_final)

                                Confirmados  Curados  Fallecidos
Casos totales en el mundo           2056054   511019      134177
Nuevos casos en el mundo (24h)        79863    36758        8193
Casos totales en España              177644    18708       70853
Nuevos casos en España (24h)           5103     3349         652


# Datos a nivel mundial

## Preparación general de datos por países

In [5]:
# Datos https://github.com/CSSEGISandData/COVID-19.git
data = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )

#Saco un dataset con solo los países
data_countries = data.groupby("Country/Region", as_index=False).sum()

# Hago transposición de los datos para mover las columnas en filas 
data_trans = data_countries.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_time = data_trans[2:]

# Cogemos solo determinados países
data_time = data_time[['China', 'France', 'Germany', 'Iran','Italy', 'Korea, South', 'Norway','Spain', 'US', 'United Kingdom']]

#Hago listado de Countries
list_countries = data_time.columns
list_countries

#Convierto la columna fecha en datetime
from datetime import date
data_time.index = pd.to_datetime(data_time.index)
data_time.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-11,83014.0,130727.0,124908.0,70029.0,152271.0,10480.0,6409.0,163027.0,526396.0,79874.0
2020-04-12,83134.0,133670.0,127854.0,71686.0,156363.0,10512.0,6525.0,166831.0,555313.0,85206.0
2020-04-13,83213.0,137875.0,130072.0,73303.0,159516.0,10537.0,6603.0,170099.0,580619.0,89570.0
2020-04-14,83306.0,131361.0,131359.0,74877.0,162488.0,10564.0,6623.0,172541.0,607670.0,94845.0
2020-04-15,83356.0,134582.0,134753.0,76389.0,165155.0,10591.0,6740.0,177644.0,636350.0,99483.0


## MUNDO: CSV : covid19_data_drive_actual.csv

In [6]:
######### Confirmados

data_confirmed = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv" )
#Saco un dataset con solo los países
data_confirmed = data_confirmed.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_confirmed = data_confirmed.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_confirmed = data_confirmed[2:]
# Cogemos solo determinados países
data_confirmed = data_confirmed[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_confirmed = data_confirmed[-1:]

######### Muertos

data_deaths = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv")
#Saco un dataset con solo los países
data_deaths = data_deaths.groupby("Country/Region", as_index=False).sum()
#Se ve que están todos y comprobado que suma
data_deaths[::]
# Hago transposición de los datos para mover las columnas en filas 
data_deaths = data_deaths.set_index('Country/Region').transpose()

# Y me quedo sólo con los datos temporales y número de casos
data_deaths = data_deaths[2:]
data_deaths.tail()
# Cogemos solo determinados países
data_deaths = data_deaths[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_deaths = data_deaths[-1:]

######### Recuperados

data_recovered = pd.read_csv("COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv" )
#Saco un dataset con solo los países
data_recovered = data_recovered.groupby("Country/Region", as_index=False).sum()
# Hago transposición de los datos para mover las columnas en filas 
data_recovered = data_recovered.set_index('Country/Region').transpose()
# Y me quedo sólo con los datos temporales y número de casos
data_recovered = data_recovered[2:]
# Cogemos solo determinados países
data_recovered = data_recovered[['China', 'France', 'Germany', 'Italy','Spain', 'US', 'United Kingdom']]
data_recovered = data_recovered[-1:]

######### Merge de datos

# Saco la fecha:
from datetime import datetime, timedelta
date_today_intern = date.today()
date_today_intern = date_today_intern.today() - timedelta(days=1) #Quito uno porque esta base de datos siempre muestra datos del día anterior
date_today_intern = date_today_intern.strftime('%d/%m')

#Juntar ahora datos
data_columns = ['Confirmados', 'Curados', 'Fallecidos']

data_together = pd.merge(data_confirmed, data_recovered, how='outer')
data_together = pd.merge(data_together, data_deaths, how='outer')
data_together.index = data_columns
data_together = data_together.transpose()
# data_together = data_together.reindex(columns =data_columns) 
# data_together = data_together.reindex(data_index)

# Exporto CSV
data_together.to_csv('covid19_data_drive_actual.csv', sep = ';')
data_together

,Confirmados,Curados,Fallecidos
Country/Region,,,
China,83356.0,78311.0,3346.0
France,134582.0,31470.0,17188.0
Germany,134753.0,72600.0,3804.0
Italy,165155.0,38092.0,21645.0
Spain,177644.0,70853.0,18708.0
US,636350.0,52096.0,28326.0
United Kingdom,99483.0,368.0,12894.0


## MUNDO: CSV : covid19_data_drive_from100.csv

In [7]:
'''Vamos a hacer un gráfico para ver los casos confirmados (desde caso 100)'''

# Coger solo datos del último mes
data_time_30days = data_time[-200:]
# Si quiero quitar China: data_time_filter = data_time_filter.drop('China', axis=1)

#Exporto los datos en diferentes arhcivos con casa país
for country in list_countries:
    dataX = data_time_30days[country]
    dataX = dataX[dataX > 200]
    dataX.to_csv('data_' + country + ".csv", index=False, header= country)

# Lista dataframes
# dataframes_list = ['dataframe_France', 'dataframe_China', 'dataframe_Germany', 'dataframe_Iran', 'dataframe_Italy', 'dataframe_Korea', 'dataframe_Norway', 'dataframe_Spain', 'dataframe_US', 'dataframe_UnitedKingdom']

# Creamos los CSV
dataframe_France = pd.read_csv('data_France.csv')
dataframe_China = pd.read_csv('data_China.csv')
dataframe_Germany = pd.read_csv('data_Germany.csv')
dataframe_Iran = pd.read_csv('data_Iran.csv')
dataframe_Italy = pd.read_csv('data_Italy.csv')
dataframe_Korea = pd.read_csv('data_Korea, South.csv')
dataframe_Norway = pd.read_csv('data_Norway.csv')
dataframe_Spain = pd.read_csv('data_Spain.csv')
dataframe_US = pd.read_csv('data_US.csv')
dataframe_UnitedKingdom = pd.read_csv('data_United Kingdom.csv')

#Les añadimos NaN al final y reindexamos
new_index = np.arange(1,51)
dataframe_France = dataframe_France.reindex(new_index)
dataframe_China = dataframe_China.reindex(new_index)
dataframe_Germany = dataframe_Germany.reindex(new_index)
dataframe_Iran = dataframe_Iran.reindex(new_index)
dataframe_Italy = dataframe_Italy.reindex(new_index)
dataframe_Korea = dataframe_Korea.reindex(new_index)
dataframe_Norway = dataframe_Norway.reindex(new_index)
dataframe_Spain = dataframe_Spain.reindex(new_index)
dataframe_US = dataframe_US.reindex(new_index)
dataframe_UnitedKingdom = dataframe_UnitedKingdom.reindex(new_index)

#Juntar dataframes y Exportar CSV completo
data_drive_from100cases = dataframe_France.join([dataframe_China, dataframe_Germany, 
                                                 dataframe_Iran, dataframe_Italy, dataframe_Korea, 
                                                 dataframe_Norway, dataframe_Spain, dataframe_US,
                                                 dataframe_UnitedKingdom])

data_drive_from100cases.to_csv('covid19_data_drive_from100.csv', sep = ';')
data_drive_from100cases.tail()

,France,China,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
46,NaN,80823.0,NaN,73303.0,147577.0,10331.0,NaN,NaN,NaN,NaN
47,NaN,80860.0,NaN,74877.0,152271.0,10384.0,NaN,NaN,NaN,NaN
48,NaN,80887.0,NaN,76389.0,156363.0,10423.0,NaN,NaN,NaN,NaN
49,NaN,80921.0,NaN,NaN,159516.0,10450.0,NaN,NaN,NaN,NaN
50,NaN,80932.0,NaN,NaN,162488.0,10480.0,NaN,NaN,NaN,NaN


## MUNDO: CSV covid19_data_drive_days.csv

In [8]:
# Coger solo datos de últimos 60 días
data_time_Xdays = data_time[-30:]

#Exportar CSV por días progresivos
data_drive_lastX = data_time_Xdays.loc[:,list_countries]
data_drive_lastX.to_csv('covid19_data_drive_days.csv', sep = ';', encoding="utf-8")
data_drive_lastX.tail()


Country/Region,China,France,Germany,Iran,Italy,"Korea, South",Norway,Spain,US,United Kingdom
2020-04-11,83014.0,130727.0,124908.0,70029.0,152271.0,10480.0,6409.0,163027.0,526396.0,79874.0
2020-04-12,83134.0,133670.0,127854.0,71686.0,156363.0,10512.0,6525.0,166831.0,555313.0,85206.0
2020-04-13,83213.0,137875.0,130072.0,73303.0,159516.0,10537.0,6603.0,170099.0,580619.0,89570.0
2020-04-14,83306.0,131361.0,131359.0,74877.0,162488.0,10564.0,6623.0,172541.0,607670.0,94845.0
2020-04-15,83356.0,134582.0,134753.0,76389.0,165155.0,10591.0,6740.0,177644.0,636350.0,99483.0


# Datos a nivel España

## ESPAÑA: Resumen datos España: data_spain_newcases24h.csv

In [9]:
path_c = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
path_d = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"
path_r = "COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv"
# Evolución últimos 20 días de neuvos casos


def day_consecutive(day):
    date_cons = date.today()
    date_cons = date_cons - timedelta(days=day)
    date_cons = date_cons.strftime('%Y-%m-%d')
    return date_cons

def data_day(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Spain']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_italy(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Italy']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_france(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['France']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def data_day_germany(data_path, number_to_rest):
    data = pd.read_csv(data_path)
    data = data.groupby("Country/Region", as_index=False).sum()
    data = data.set_index("Country/Region").transpose()
    data = data['Germany']
    data = int(data.iloc[number_to_rest]) - int(data.iloc[number_to_rest-1])
    return data

def create_dataframe(data_day_name, data_path, index_title):
    dataframe = pd.DataFrame({day_consecutive(40): data_day_name(data_path, -40),
                              day_consecutive(39): data_day_name(data_path, -39),
                              day_consecutive(38): data_day_name(data_path, -38),
                              day_consecutive(37): data_day_name(data_path, -37),
                              day_consecutive(36): data_day_name(data_path, -36),
                              day_consecutive(35): data_day_name(data_path, -35),
                              day_consecutive(34): data_day_name(data_path, -34),
                              day_consecutive(33): data_day_name(data_path, -33),
                              day_consecutive(32): data_day_name(data_path, -32),
                              day_consecutive(31): data_day_name(data_path, -31),
                              day_consecutive(30): data_day_name(data_path, -30),
                              day_consecutive(29): data_day_name(data_path, -29),
                              day_consecutive(28): data_day_name(data_path, -28),
                              day_consecutive(27): data_day_name(data_path, -27),
                              day_consecutive(26): data_day_name(data_path, -26),
                              day_consecutive(25): data_day_name(data_path, -25),
                              day_consecutive(24): data_day_name(data_path, -24),
                              day_consecutive(23): data_day_name(data_path, -23),
                              day_consecutive(22): data_day_name(data_path, -22),
                              day_consecutive(21): data_day_name(data_path, -21),
                              day_consecutive(20): data_day_name(data_path, -20),
                              day_consecutive(19): data_day_name(data_path, -19), 
                              day_consecutive(18): data_day_name(data_path, -18),
                              day_consecutive(17): data_day_name(data_path, -17),
                              day_consecutive(16): data_day_name(data_path, -16),
                              day_consecutive(15): data_day_name(data_path, -15),
                              day_consecutive(14): data_day_name(data_path, -14),
                              day_consecutive(13): data_day_name(data_path, -13),
                              day_consecutive(12): data_day_name(data_path, -12),
                              day_consecutive(11): data_day_name(data_path, -11),
                              day_consecutive(10): data_day_name(data_path, -10),
                              day_consecutive(9): data_day_name(data_path, -9),
                              day_consecutive(8): data_day_name(data_path, -8),
                              day_consecutive(7): data_day_name(data_path, -7),
                              day_consecutive(6): data_day_name(data_path, -6),
                              day_consecutive(5): data_day_name(data_path, -5),
                              day_consecutive(4): data_day_name(data_path, -4),
                              day_consecutive(3): data_day_name(data_path, -3),
                              day_consecutive(2): data_day_name(data_path, -2),
                              day_consecutive(1): data_day_name(data_path, -1),
                             }, index=index_title)
                                    
    return dataframe

data_spain_confirmed24h = create_dataframe(data_day, path_c, ['Confirmados (24h) en España'])
data_spain_deaths24h = create_dataframe(data_day, path_d, ['Fallecidos (24h) en España'])
data_spain_recovered24h = create_dataframe(data_day, path_r, ['Recuperados (24h) en España'])

#Juntar dataframes
data_spain_newcases24h = pd.concat([data_spain_confirmed24h,data_spain_recovered24h,data_spain_deaths24h], axis=0)
data_spain_newcases24h.to_csv('data_spain_newcases24h.csv', sep = ';')
data_spain_deaths24h.to_csv('data_spain_newcases24h_deaths.csv', sep = ';')

######## Lo mismo en italia
data_italy_confirmed24h = create_dataframe(data_day_italy, path_c, ['Confirmados (24h) en Italia'])
data_italy_deaths24h = create_dataframe(data_day_italy, path_d, ['Fallecidos (24h) en Italia'])
data_italy_recovered24h = create_dataframe(data_day_italy, path_r, ['Recuperados (24h) en Italia'])
data_italy_newcases24h = pd.concat([data_italy_confirmed24h,data_italy_recovered24h,data_italy_deaths24h], axis=0)

######## Lo mismo en Francia
data_france_confirmed24h = create_dataframe(data_day_france, path_c, ['Confirmados (24h) en Francia'])
data_france_deaths24h = create_dataframe(data_day_france, path_d, ['Fallecidos (24h) en Francia'])
data_france_recovered24h = create_dataframe(data_day_france, path_r, ['Recuperados (24h) en Francia'])
data_france_newcases24h = pd.concat([data_france_confirmed24h,data_france_recovered24h,data_france_deaths24h], axis=0)

######## Lo mismo en Alemania
data_germany_confirmed24h = create_dataframe(data_day_germany, path_c, ['Confirmados (24h) en Alemania'])
data_germany_deaths24h = create_dataframe(data_day_germany, path_d, ['Fallecidos (24h) en Alemania'])
data_germany_recovered24h = create_dataframe(data_day_germany, path_r, ['Recuperados (24h) en Alemania'])
data_germany_newcases24h = pd.concat([data_germany_confirmed24h,data_germany_recovered24h,data_germany_deaths24h], axis=0)


###### Juntar dataframes Spain + Italy sólo de fallecidos
data_somecountries_newcases24h_deaths = pd.concat([data_spain_deaths24h, data_italy_deaths24h,
                                                   data_france_deaths24h, data_germany_deaths24h], axis=0)
data_somecountries_newcases24h_deaths.to_csv('data_somecountries_newcases24h.csv', sep = ';')
data_somecountries_newcases24h_deaths


,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,...,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15
Fallecidos (24h) en España,5,7,11,7,19,1,78,62,94,53,...,700,704,747,655,634,525,603,547,300,652
Fallecidos (24h) en Italia,36,133,97,168,196,0,439,175,368,349,...,636,604,542,610,570,619,431,566,602,578
Fallecidos (24h) en Francia,2,8,0,14,15,0,31,12,0,58,...,833,1417,544,1341,987,636,561,574,762,1440
Fallecidos (24h) en Alemania,0,0,2,0,1,0,4,2,2,6,...,226,206,333,258,160,-31,286,172,100,510


## ESPAÑA: CSV: covid19_spain_ccaa.csv datos principales

In [38]:
#Exportar CSV
data_spain_ccaa.to_csv('covid19_spain_ccaa.csv', sep = ';', encoding="utf-8")
data_spain_ccaa.head(19)

,Confirmados,Curados,Fallecidos,Nuevos casos (24h)
Comunidades Autónomas,,,,
Andalucía,10595,2224,865,289
Aragón,4338,1012,514,93
Asturias,2096,487,166,45
Baleares,1606,881,125,35
Canarias,1975,622,104,17
Cantabria,1823,363,132,27
Castilla La Mancha,14680,2998,1755,351
Castilla y León,13697,4521,1337,517
Cataluña,36505,16651,3756,1308


## ESPAÑA: CSV: covid19_spain_ccaa_lineal.csv

In [39]:
# Casos confirmados por ccaa temporal

# Cojo la fila de confirmados que me interesa
data_spain_ccaa_last = data_spain_ccaa.copy()
data_spain_ccaa_last.rename(columns = {'Confirmados': date_today_large}, index={'ccaa': 'Comunidades Autónomas'}, inplace = True)
data_spain_ccaa_last = data_spain_ccaa_last[date_today_large]
data_spain_ccaa_last = pd.to_numeric(data_spain_ccaa_last).astype(np.int64)
data_spain_ccaa_last.head()

#HACER MERGE SI NO ESTÁN ESOS DATOS

# Importo los últimos datos recogidos acumulados anteriormente
data_process = pd.read_csv("covid19_spain_ccaa_evolution2.csv", sep = ';', encoding="utf-8")
data_process = data_process.set_index('Comunidades Autónomas')

# Hacer merge de ambos dataframes, con un condicional comprobando si es necesario
if data_process.columns[-1] == data_spain_ccaa_last.name:
    print('La columna ya está: NO hacer merge y no se exporta tabla')
else:
    print('La última columna es diferente y no está: SE HACE MERGE')
    data_process = pd.merge(data_process,data_spain_ccaa_last, left_index=True, right_index=True)
    #Exportar a CSV
    data_process.to_csv('covid19_spain_ccaa_evolution2.csv', sep = ';', encoding="utf-8", index=True, index_label='Comunidades Autónomas')

data_process

La última columna es diferente y no está: SE HACE MERGE


,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,...,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15
Comunidades Autónomas,,,,,,,,,,,,,,,,,,,,,
Andalucía,6,12,13,13,13,21,21,21,54,71,...,8581,8767,8997,9261,9510,9712,10006,10187,10192,10595
Aragón,2,2,2,2,2,6,6,6,13,38,...,3347,3449,3549,3685,3831,3969,4070,4187,4245,4338
Asturias,0,1,1,2,5,5,5,5,7,22,...,1646,1679,1705,1737,1799,1827,1892,1958,2051,2096
Baleares,2,2,2,5,6,6,6,6,11,13,...,1320,1369,1412,1448,1488,1507,1534,1550,1571,1606
Canarias,6,7,7,7,8,11,11,11,22,25,...,1649,1725,1762,1834,1858,1887,1918,1944,1958,1975
Cantabria,0,10,10,10,10,10,10,10,12,12,...,1483,1501,1572,1619,1659,1719,1752,1777,1796,1823
Castilla La Mancha,0,3,7,12,13,15,15,15,26,39,...,10602,11077,11788,12489,13063,13456,13698,14054,14329,14680
Castilla y León,2,3,5,11,11,14,14,14,23,56,...,9116,9581,10058,10518,11102,11543,12118,12628,13180,13697
Cataluña,3,15,15,15,24,24,24,24,75,124,...,26824,28323,29647,31043,31824,32984,34027,34726,35197,36505


## ESPAÑA: Análisis rangos de esas: CSV

In [40]:
data_spain_ccaa_age = pd.read_csv("https://raw.githubusercontent.com/datadista/datasets/master/COVID%2019/nacional_covid19_rango_edad.csv", 
                                  sep = ',', encoding="utf-8")
data_spain_ccaa_age = data_spain_ccaa_age[data_spain_ccaa_age["fecha"] == date_today_large]

def take_edad(sexo):
    data_spain_ccaa_age_all = data_spain_ccaa_age[data_spain_ccaa_age["sexo"] == sexo]
    #Para sacar datos total
    data_spain_ccaa_age_total = data_spain_ccaa_age_all.tail(1)
    data_spain_ccaa_age_total = data_spain_ccaa_age_total.iloc[:, [3,4,6]]
    data_spain_ccaa_age_total.columns = ['Confirmados','Hospitalizados', 'Fallecidos']
    #Para sacar datos concretos (de ambos, hombres y mujeres)
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_all[:-1]
    data_spain_ccaa_age_concrete = data_spain_ccaa_age_concrete.iloc[:, [1,3,4,6]]
    data_spain_ccaa_age_concrete.columns = ['Rango de edad','Confirmados','Hospitalizados', 'Fallecidos']
    data_spain_ccaa_age_concrete.set_index('Rango de edad', inplace=True)
    return data_spain_ccaa_age_concrete, data_spain_ccaa_age_total


data_spain_ccaa_age_ambos_concrete, data_spain_ccaa_age_ambos_total = take_edad('ambos')
data_spain_ccaa_age_hombres_concrete, data_spain_ccaa_age_hombres_total = take_edad('hombres')
data_spain_ccaa_age_mujeres_concrete, data_spain_ccaa_age_mujeres_total = take_edad('mujeres')


data_spain_ccaa_age_ambos_total.index = ['Total ambos sexos']
data_spain_ccaa_age_hombres_total.index = ['Total hombres']
data_spain_ccaa_age_mujeres_total.index = ['Total mujeres']
data_spain_ccaa_age_ambos_concat = pd.concat([data_spain_ccaa_age_mujeres_total, data_spain_ccaa_age_hombres_total, 
                                             data_spain_ccaa_age_ambos_total], axis=0)

data_spain_ccaa_age_mujeres_concrete.to_csv('covid19_spain_ccaa_ages_mujeres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_hombres_concrete.to_csv('covid19_spain_ccaa_ages_hombres.csv', sep = ';', encoding="utf-8")
data_spain_ccaa_age_ambos_concat.to_csv('covid19_spain_ccaa_ages_ambos.csv', sep = ';', encoding="utf-8")

data_spain_ccaa_age_mujeres_concrete

,Confirmados,Hospitalizados,Fallecidos
Rango de edad,,,
0-9,177,57,1
10-19,361,73,1
20-29,4094,463,5
30-39,7125,1136,15
40-49,10236,2202,39
50-59,12409,3562,89
60-69,9332,4515,264
70-79,8069,5434,778
80-89,8830,4938,1653


# Subir archivos a SERVIDOR desde FTP

In [6]:
import csv
from ftplib import FTP
from io import StringIO

In [7]:
# Ayuda: https://www.pythoncircle.com/post/668/uploading-a-file-to-ftp-server-using-python/

passw = pd.read_csv("pass.txt", sep = ',', encoding="utf-8")
host = "akakicreations.com"
username = passw.columns[0]
password = passw.columns[1]

ftp = FTP(host=host)

login_status = ftp.login(user=username, passwd=password)
print(login_status)

print(ftp.dir())
fp = open("covid19_data_drive_days.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_days.csv"), fp, 1024)
fp = open("covid19_data_drive_from100.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_from100.csv"), fp, 1024)
fp = open("covid19_spain_ccaa.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_evolution2.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_evolution2.csv"), fp, 1024)
fp = open("covid19_data_drive_actual.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_actual.csv"), fp, 1024)
fp = open("covid19_data_drive_resume.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_data_drive_resume.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_mujeres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_mujeres.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_hombres.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_hombres.csv"), fp, 1024)
fp = open("covid19_spain_ccaa_ages_ambos.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("covid19_spain_ccaa_ages_ambos.csv"), fp, 1024)
fp = open("data_spain_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h.csv"), fp, 1024)
fp = open("data_spain_newcases24h_deaths.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_spain_newcases24h_deaths.csv"), fp, 1024)
fp = open("data_somecountries_newcases24h.csv", 'rb')
ftp.storbinary('STOR %s' % os.path.basename("data_somecountries_newcases24h.csv"), fp, 1024)
fp.close()


230-OK. Current restricted directory is /
230 318 Kbytes used (0%) - authorized: 51200 Kb
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 11 11:51 .
drwxr-xr-x    2 akakicre   akakicre         4096 Apr 11 11:51 ..
-rw-------    1 akakicre   akakicre           10 Apr 15 15:35 .ftpquota
-rw-r--r--    1 akakicre   akakicre       260069 Apr  3 13:34 Imagen_github.jpg
-rw-r--r--    1 akakicre   akakicre          266 Apr 15 15:35 covid19_data_drive_actual.csv
-rw-r--r--    1 akakicre   akakicre         2807 Apr 15 15:35 covid19_data_drive_days.csv
-rw-r--r--    1 akakicre   akakicre         3607 Apr 15 15:35 covid19_data_drive_from100.csv
-rw-r--r--    1 akakicre   akakicre          219 Apr 15 15:35 covid19_data_drive_resume.csv
-rw-r--r--    1 akakicre   akakicre          591 Apr 15 15:35 covid19_spain_ccaa.csv
-rw-r--r--    1 akakicre   akakicre          235 Apr 14 12:39 covid19_spain_ccaa_ages.csv
-rw-r--r--    1 akakicre   akakicre          137 Apr 15 15:35 covid19_spain_ccaa_ages_a